# Import Project

## Google Colab

In [ ]:
import os
import sys
import torch

!git clone https://github.com/AnasShahzad1996/novel_view_synthesis.git

Cloning into 'novel_view_synthesis'...
remote: Enumerating objects: 551, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 551 (delta 24), reused 74 (delta 18), pack-reused 467
Receiving objects: 100% (551/551), 38.77 MiB | 23.18 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [ ]:
!cd novel_view_synthesis && git checkout donerf

Branch 'donerf' set up to track remote branch 'donerf' from 'origin'.
Switched to a new branch 'donerf'


In [ ]:
project_path = '/content/novel_view_synthesis/src/src'
os.chdir(project_path)
sys.path.insert(1, project_path)
print('CUDA availability:', torch.cuda.is_available())

CUDA availability: True


In [ ]:
!nvidia-smi

Mon Jul 24 12:02:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Project Setup

In [ ]:
!cd .. && python -m pip install -r requirements.txt

## Install kilonerf_cuda

In [ ]:
!apt install libgl-dev freeglut3-dev

### Using Pre-compiled Wheel

In [ ]:
!pip install cuda/dist/kilonerf_cuda-0.0.0-cp38-cp38-linux_x86_64.whl

### Or Alternatively

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.1.1/local_installers/cuda_11.1.1_455.32.00_linux.run
!sh cuda_11.1.1_455.32.00_linux.run --silent
!echo -e "\nexport PATH=\"/usr/local/cuda/bin:\$PATH\"" >> ~/.bashrc
!echo "export LD_LIBRARY_PATH=\"/usr/local/cuda/lib64:\$LD_LIBRARY_PATH\"" >> ~/.bashrc

In [ ]:
!apt install gfortran libopenblas-dev
!wget http://icl.utk.edu/projectsfiles/magma/downloads/magma-2.5.4.tar.gz
!tar -zxvf magma-2.5.4.tar.gz

In [ ]:
!cp magma-2.5.4/make.inc-examples/make.inc.openblas magma-2.5.4/make.inc

In [ ]:
%env GPU_TARGET="Maxwell Pascal Volta Turing Ampere"
%env CUDADIR=/usr/local/cuda
%env OPENBLASDIR=/usr/

env: GPU_TARGET="Maxwell Pascal Volta Turing Ampere"
env: CUDADIR=/usr/local/cuda
env: OPENBLASDIR=/usr/


In [ ]:
!cd magma-2.5.4 && make
!cd magma-2.5.4 && sudo -E make install prefix=/usr/local/magma

In [ ]:
!cd cuda && python setup.py develop

# Dataset

In [ ]:
!mkdir data

In [ ]:
!cd data && wget https://repository.tugraz.at/records/jjs3x-4f133/files/classroom.zip && unzip -n classroom.zip

--2023-07-24 12:03:11--  https://repository.tugraz.at/records/jjs3x-4f133/files/classroom.zip
Resolving repository.tugraz.at (repository.tugraz.at)... 129.27.2.161
Connecting to repository.tugraz.at (repository.tugraz.at)|129.27.2.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689090205 (657M) [application/octet-stream]
Saving to: ‘classroom.zip’

classroom.zip        21%[===>                ] 144.55M   358KB/s    in 5m 20s  

2023-07-24 12:08:35 (463 KB/s) - Connection closed at byte 151574166. Retrying.

--2023-07-24 12:08:36--  (try: 2)  https://repository.tugraz.at/records/jjs3x-4f133/files/classroom.zip
Connecting to repository.tugraz.at (repository.tugraz.at)|129.27.2.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689090205 (657M) [application/octet-stream]
Saving to: ‘classroom.zip’

classroom.zip         0%[                    ]   1.03M   160KB/s    in 10s     

2023-07-24 12:08:49 (104 KB/s) - Connection closed at b

# DONeRF Experiment
## Train

In [ ]:
!python train.py -c ../configs/DONeRF_8_samples.ini --data data/classroom/ --logDir data/classroom/output_results/ --device 0 --storeFullData

device Tesla T4
WARNING! - import of cuda kernels for 'disc_depth_multiclass' failed - falling back to PyTorch
Loading dataset train: 100% 210/210 [00:08<00:00, 24.31it/s]
Loading dataset val  : 100% 30/30 [00:01<00:00, 25.05it/s]
Loading dataset test : 100% 60/60 [00:02<00:00, 25.13it/s]
Training config: lo_l1.0_0.0_SpPoDir[128]-relu0(256x8)-CD-128-5-5_l1.0_10.0_RayMarchFromPoses_nSD[2_LSfCD_128_0.0](nerf(10-4))-relu1(256x80..63-7.63.)-RGBARayMarch (../configs/DONeRF_2_samples.ini)
Reloading Checkpoint from /data/output_results/classroom/lo_l1.0_0.0_SpPoDir[128]-relu0(256x8)-CD-128-5-5_l1.0_10.0_RayMarchFromPoses_nSD[2_LSfCD_128_0.0](nerf(10-4))-relu1(256x80..63-7.63.)-RGBARayMarch/relu0(256x8)_0060000.weights
Reloading optimizer checkpoint from /data/output_results/classroom/lo_l1.0_0.0_SpPoDir[128]-relu0(256x8)-CD-128-5-5_l1.0_10.0_RayMarchFromPoses_nSD[2_LSfCD_128_0.0](nerf(10-4))-relu1(256x80..63-7.63.)-RGBARayMarch/relu0(256x8)_0060000.optimizer
Reloading Checkpoint from /data/ou

## Test

In [ ]:
!python test.py -c ../configs/DONeRF_8_samples.ini --data data/classroom/ --logDir data/output_results/ --device 0 --storeFullData --camPath cam_path_rotate --outputVideoName cam_path_rotate --videoFrames 300

# NeRF Experiment
## Train

In [ ]:
!python train.py -c ../configs/NeRF_coarse_only_8_samples_log_warp.ini --data data/classroom/ --logDir data/classroom/output_results/ --device 0 --storeFullData

## Test

In [ ]:
!python test.py -c ../configs/NeRF_coarse_only_8_samples_log_warp.ini --data data/classroom/ --logDir data/output_results/ --device 0 --storeFullData --camPath cam_path_rotate --outputVideoName cam_path_rotate --videoFrames 300

# KiloNeRF Experiment
## Train

In [ ]:
!python train.py -c ../configs/KiloNeRF_8_samples_log_warp.ini --data data/classroom/ --logDir data/classroom/output_results/ --device 0 --storeFullData

## Test

In [ ]:
!python test.py -c ../configs/KiloNeRF_8_samples_log_warp.ini --data data/classroom/ --logDir data/output_results/ --device 0 --storeFullData --camPath cam_path_rotate --outputVideoName cam_path_rotate --videoFrames 300